### TRANSFER LEARNING LECTURE NOTES

In [ ]:
import numpy as np
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image

# Load the ResNet50 model pre-trained on ImageNet
model = ResNet50(weights='imagenet')

# Function to classify an image
def classify_image(img_path):
    
    
    # Load the image with the target size of 224x224
    img = image.load_img(img_path, target_size=(224, 224))
    # Convert the image to a numpy array
    img_array = image.img_to_array(img)
    
    
    # Expand dimensions to match the shape the model expects
    img_array = np.expand_dims(img_array, axis=0) 
    # shape=(1, 224, 224, 3) because we have one image , why ? because the model expects a batch of images
    
    
    # Preprocess the image
    img_array = preprocess_input(img_array) # example: normalize the image
    # Make predictions
    preds = model.predict(img_array)
    # Decode the predictions
    decoded_preds = decode_predictions(preds, top=3)[0]
    return decoded_preds

# Example usage
img_path = 'elephant.jpg'
predictions = classify_image(img_path)
for pred in predictions:
    print(f"Predicted: {pred[1]} with probability {pred[2]:.4f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
35363/35363 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Predicted: African_elephant with probability 0.7037
Predicted: tusker with probability 0.1986
Predicted: Indian_elephant with probability 0.0977


Now we fine-tune the model to classify the images of the new dataset.

Fine-tuning is the process of training the pre-trained model on a new dataset to improve performance.

We take the output of the last layer of the base model and pass it to a new model with the same architecture but with a new output layer.

The convolution to MLP (Multi-Layer Perceptron) bridge is done by the flatten layer, which converts the feature map to a vector.

Alternatively, we can use the global average pooling layer, which reduces the spatial dimensions of the feature map to a single value.

In [ ]:
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

base_model = ResNet50(weights='imagenet', include_top=False) # load the ResNet50 model without the top layer

# add a global average pooling layer 
x = base_model.output
x = GlobalAveragePooling2D()(x) # reduce the spatial dimensions of the feature map to a single value
x = Dense(1024, activation='relu')(x) # add a fully connected layer

# Add a logistic layer with 200 classes
predictions = Dense(200, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:

# with the sequential way we can add layers one by one : 

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense

base_model = ResNet50(weights='imagenet', include_top=False) # load the ResNet50 model without the top layer

model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(1024, activation='relu'))
model.add(Dense(200, activation='softmax'))

# Freeze the base model (the first 175 layers)
for layer in base_model.layers[:175]:
    layer.trainable = False

# Compile the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

# train the model
model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

